In [570]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import pandas as pd
import ast

In [599]:
samples = 1000

In [572]:
model_name = "Isotonic/deberta-v3-base_finetuned_ai4privacy_v2"

### Load email dataset

In [573]:
def load_data_csv_file(csv_file_name):
    df = pd.read_csv(csv_file_name)
    return df

In [574]:
df = load_data_csv_file('../dataset/email_dataset_labelled.csv')

In [548]:
df.head()

,Body,Priavte data,Unnamed: 2,Unnamed: 3
0,Here is our forecast,[],NaN,NaN
1,Traveling to have a business meeting takes the...,['Austin'],NaN,NaN
2,test successful. way to go!!!,[],NaN,NaN
3,"Randy,\n Can you send me a schedule of the sa...","['Randy', 'Patti S', 'Phillip']",NaN,NaN
4,Let's shoot for Tuesday at 11:45.,"['Tuesday', '11:45']",NaN,NaN


### Load dataset from hugging face

In [600]:
dataset_name = "ai4privacy/pii-masking-300k"

In [601]:
def load_data(dataset_name):
    dataset = load_dataset(dataset_name)
    filtered_dataset = dataset.filter(lambda example: example['language'] == 'English')
    df = filtered_dataset['train'].to_pandas() 
    df.head()
    return df

In [604]:
df = load_data(dataset_name=dataset_name)

In [605]:
len(df)

29908

In [84]:
df.head(10)

,source_text,target_text,privacy_mask,span_labels,mbert_text_tokens,mbert_bio_labels,id,language,set
0,Subject: Group Messaging for Admissions Proces...,Subject: Group Messaging for Admissions Proces...,"[{'value': 'wynqvrh053', 'start': 287, 'end': ...","[[440, 453, ""USERNAME""], [430, 437, ""TIME""], [...","[Sub, ##ject, :, Group, Mess, ##aging, for, Ad...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",40767A,English,train
1,- Meeting at 2:33 PM\n- N23 - Meeting at 11:29...,- Meeting at [TIME]\n- [USERNAME] - Meeting at...,"[{'value': '2:33 PM', 'start': 13, 'end': 20, ...","[[74, 81, ""TIME""], [50, 60, ""USERNAME""], [40, ...","[-, Meeting, at, 2, :, 33, PM, -, N, ##23, -, ...","[O, O, O, B-TIME, I-TIME, I-TIME, I-TIME, O, O...",40767B,English,train
2,Subject: Admission Notification - Great Britai...,Subject: Admission Notification - Great Britai...,"[{'value': '5:24am', 'start': 263, 'end': 269,...","[[395, 407, ""SOCIALNUMBER""], [358, 375, ""EMAIL...","[Sub, ##ject, :, Ad, ##mission, Not, ##ificati...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",40768A,English,train
3,Card: KB90324ER\n Country: GB\n Building: ...,Card: [IDCARD]\n Country: [COUNTRY]\n Buil...,"[{'value': 'KB90324ER', 'start': 6, 'end': 15,...","[[390, 393, ""STATE""], [368, 378, ""CITY""], [346...","[Card, :, KB, ##90, ##32, ##4, ##ER, \, n, Cou...","[O, O, B-IDCARD, I-IDCARD, I-IDCARD, I-IDCARD,...",40768B,English,train
4,"N, WA14 5RW\n Password: r]iD1#8\n\n...and so...","N, WA14 5RW\n Password: [PASS]\n\n...and so ...","[{'value': 'r]iD1#8', 'start': 26, 'end': 33, ...","[[336, 352, ""DATE""], [26, 33, ""PASS""]]","[N, ,, W, ##A, ##14, 5, ##R, ##W, \, n, Pass, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-PASS...",40768C,English,train
5,Subject: Admission Application Attachments Con...,Subject: Admission Application Attachments Con...,"[{'value': '301025226', 'start': 311, 'end': 3...","[[311, 320, ""PASSPORT""]]","[Sub, ##ject, :, Ad, ##mission, Application, A...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",40769A,English,train
6,981\n- Social Security Number: 554.575.9355\n-...,981\n- Social Security Number: [SOCIALNUMBER]\...,"[{'value': '554.575.9355', 'start': 30, 'end':...","[[263, 279, ""TEL""], [226, 249, ""EMAIL""], [206,...","[981, -, Social, Security, Number, :, 554, ., ...","[O, O, O, O, O, O, O, O, B-SOCIALNUMBER, I-SOC...",40769B,English,train
7,s carefully and inform us immediately if there...,s carefully and inform us immediately if there...,"[{'value': 'Rue des Écoles', 'start': 320, 'en...","[[342, 354, ""DATE""], [338, 340, ""TIME""], [320,...","[s, care, ##fully, and, info, ##rm, us, immedi...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",40769C,English,train
8,"- id_1:\n Feb 8, 1986\n iloweintögl\n 4929-...",- id_1:\n [BOD]\n [USERNAME]\n [TEL]\n Det...,"[{'value': 'Feb 8, 1986', 'start': 10, 'end': ...","[[287, 300, ""TEL""], [280, 284, ""USERNAME""], [2...","[-, id, _, 1, :, Feb, 8, ,, 1986, il, ##owe, #...","[O, O, O, O, O, O, B-BOD, I-BOD, I-BOD, I-BOD,...",40772A,English,train
9,y involved in community service initiatives.\n...,y involved in community service initiatives.\n...,"[{'value': 'Apr 29, 1973', 'start': 55, 'end':...","[[339, 355, ""TEL""], [314, 336, ""USERNAME""], [2...","[y, involved, in, community, service, initiati...","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-BOD,...",40772B,English,train


In [606]:
def filtered_dataset(rows, df):
    return df.head(rows)

In [607]:
df_filtered = filtered_dataset(samples, df)

In [451]:
df_filtered['Priavte data'][4]

"['Tuesday', '11:45']"

In [608]:
len(df_filtered)

1000

In [609]:
predicted_sensitive_values = []
true_sensitive_values = []
all_words =[]

In [610]:
import re
def clean_text(text):
    text = text.replace('\\n', ' ')  # Replace literal '\n' with space
    text = re.sub(r'\s+', ' ', text).strip()  # Replace any extra spaces with a single space
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    text = text.replace("**", ' ')
    return text

### Extraction for email dataset

In [579]:
sentence = df_filtered['Body'][9]
print(sentence)

---------------------- Forwarded by Phillip K Allen/HOU/ECT on 10/16/2000 
 01:42 PM ---------------------------
 "Buckner, Buck" <buck.buckner@honeywell.com> on 10/12/2000 01:12:21 PM
 To: "'Pallen@Enron.com'" <Pallen@Enron.com>
 cc:  
 Subject: FW: fixed forward or other Collar floor gas price terms
 Phillip,
 > As discussed during our phone conversation, In a Parallon 75 microturbine
 > power generation deal for a national accounts customer, I am developing a
 > proposal to sell power to customer at fixed or collar/floor price. To do
 > so I need a corresponding term gas price for same. Microturbine is an
 > onsite generation product developed by Honeywell to generate electricity
 > on customer site (degen). using natural gas. In doing so, I need your
 > best fixed price forward gas price deal for 1, 3, 5, 7 and 10 years for
 > annual/seasonal supply to microturbines to generate fixed kWh for
 > customer. We have the opportunity to sell customer kWh 's using
 > microturbine or sell 

In [580]:
clean_text_sentence = clean_text(df_filtered['Body'][9])
print(clean_text_sentence)

---------------------- Forwarded by Phillip K Allen/HOU/ECT on 10/16/2000 01:42 PM --------------------------- "Buckner, Buck" <buck.buckner@honeywell.com> on 10/12/2000 01:12:21 PM To: "'Pallen@Enron.com'" <Pallen@Enron.com> cc: Subject: FW: fixed forward or other Collar floor gas price terms Phillip, > As discussed during our phone conversation, In a Parallon 75 microturbine > power generation deal for a national accounts customer, I am developing a > proposal to sell power to customer at fixed or collar/floor price. To do > so I need a corresponding term gas price for same. Microturbine is an > onsite generation product developed by Honeywell to generate electricity > on customer site (degen). using natural gas. In doing so, I need your > best fixed price forward gas price deal for 1, 3, 5, 7 and 10 years for > annual/seasonal supply to microturbines to generate fixed kWh for > customer. We have the opportunity to sell customer kWh 's using > microturbine or sell them turbines thems

In [581]:
import json

In [582]:
def get_true_labels(df_filtered):
    for i in range(len(df_filtered)):
        true_label_array = ast.literal_eval(df_filtered['Priavte data'][i])
        true_sensitive_values.append(true_label_array)


In [583]:
def get_all_words(source_texts):
    for text in source_texts:
        words = []
        sentence = clean_text(text)
        words = sentence.split()
        all_words.append(words)

In [584]:
predicted_sensitive_values = []
true_sensitive_values = []
all_words =[]

get_all_words(df_filtered['Body'])
get_true_labels(df_filtered)

In [585]:
true_sensitive_values

[[],
 ['Austin'],
 [],
 ['Randy', 'Patti S', 'Phillip'],
 ['Tuesday', '11:45'],
 ['Philip', 'Greg'],
 ['Phillip Allen',
  'pallen@enron.com',
  'Mike Grigsby',
  'mike.grigsby@enron.com',
  'Keith Holst',
  'kholst@enron.com',
  'Monique Sanchez',
  'Frank Ermis',
  'John Lavorato'],
 ['10', '11:30'],
 ['pallen',
  'ke9davis',
  '64.216.90.105',
  '255.255.255.248',
  '64.216.90.110',
  '151.164.1.8',
  '0413',
  '105891'],
 ['Phillip K Allen',
  'Buckner, Buck',
  'buck.buckner@honeywell.com',
  'pallen@Enron.com',
  'Sempra energy gas trading',
  'San Diego',
  'San Diego Gas & Electric',
  'Buck Buckner',
  'P.E., MBA',
  'Manager, Business Development and Planning',
  'Big Box Retail Sales',
  'Honeywell Power Systems, Inc.',
  '8725 Pan American Frwy',
  'Albuquerque, NM 87113',
  '505-798-6424',
  '505-798-6050x',
  '505-220-4129',
  '888/501-3145',
  '10/16/2000',
  '01:42 PM',
  '10/12/2000 01:12:21 PM',
  'Parallon 75 microturbine'],
 ['Mr. Buckner',
  'San Diego',
  'Enron En

### Extraction for huggingface dataset 

In [611]:
def get_all_words(source_texts):
    for text in source_texts:
        words = []
        sentence = clean_text(text)
        words = sentence.split()
        all_words.append(words)

In [612]:
def get_true_labels(df_filtered):
    for i in range(len(df_filtered)):
        label = [item["value"] for item in df_filtered['privacy_mask'][i]]
        label = ["".join(value.split()) for value in label]
        true_sensitive_values.append(label)

In [613]:
predicted_sensitive_values = []
true_sensitive_values = []
all_words =[]

get_all_words(df_filtered['source_text'])
get_true_labels(df_filtered)

In [614]:
len(true_sensitive_values)

1000

Prediction using Hugging face model - 

In [586]:
def load_model(model_name):
    # Load pre-trained BERT tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForTokenClassification.from_pretrained(model_name)
    return tokenizer, model

In [587]:
tokenizer, model = load_model(model_name)

In [589]:
def run_predictions(source_texts):
    
    for text in source_texts:
        prediction = []
        ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="average")
        sentence = clean_text(text)

        results = ner_pipeline(sentence)

        # Print results
        for entity in results:
            # print(f"Entity: {entity['word']}, Label: {entity['entity_group']}, Score: {entity['score']:.4f}")
            prediction.append(entity['word'])
        
        predicted_sensitive_values.append(prediction)

        

In [590]:
predicted_sensitive_values = []
run_predictions(df_filtered['Body'])

Device set to use cpu


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
c:\Users\neham\TAMU\research_project_llm\Spring2025\private-data-blocker\.venv\Lib\site-packages\transformers\pipelines\token_classification.py:398: UserWarning: Tokenizer does not support real words, using fallback heuristic
  warnings.warn(
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu


In [591]:
predicted_sensitive_values

[[],
 [],
 [],
 ['Phillip'],
 ['11:45.'],
 ['Phillip'],
 ['PhillipAllen(pallen@enron.com)',
  'MikeGrigsby(mike.grigsby@enron.com)',
  'KeithHolst(kholst@enron.com)',
  'Monique',
  'Sanchez',
  'Frank',
  'Ermis',
  'John',
  'Lavorato',
  'Phillip',
  'Allen'],
 [],
 ['pallen',
  'ke9davis',
  '64.216.90.105',
  '255.255.255.248',
  '64.216.90.110',
  '151.164.1.8',
  '0413',
  '105891'],
 ['Phillip',
  'Allen/HOU/ECT',
  '10/16/200001:42PM',
  '---------------------------',
  'Buck"<buck.buckner@honeywell.com>',
  '10/12/200001:12:21PM',
  '"\'Pallen@Enron.com\'"<Pallen@Enron.com>',
  'nationalaccounts',
  'Sempra',
  'San',
  '-302',
  'Buck',
  'Honeywell',
  '8725',
  'PanAmericanFrwy',
  '87113',
  '505-798-6424',
  '>',
  '505-798-6050x',
  '505-220-4129',
  '888/501-3145'],
 ['Mr.', 'San', 'Enron', 'Zarin', 'Imam', '713-853-7107.', 'Phillip', 'Allen'],
 ['Phillip'],
 ['Phillip',
  'Allen/HOU/ECT',
  '10/09/200002:16PM',
  '---------------------------',
  'Burchfield',
  '10/06

In [594]:
len(all_words)

100

In [595]:
def get_measures(true_sensitive_values, predicted_sensitive_values, all_words):
    metrics_mapping = []
    for true_list, pred_list, all_word_list in zip(true_sensitive_values, predicted_sensitive_values, all_words):
        tp = 0
        tn=0
        fp = 0 
        fn =0
        for true_value in true_list:
            # if true_value in pred_list:
            if any(true_value in pred_value for pred_value in pred_list):
                tp += 1  # Correctly identified (True Positive)
            else:
                print(f"Missed entity (False Negative): {true_value}")
                fn += 1  # Missed entity

        # Check for False Positives
        for predicted_value in pred_list:
            # if predicted_value not in true_list:
            if not any(true_value in predicted_value for true_value in true_list):
                print(f"Incorrectly identified (False Positive): {predicted_value}")
                fp += 1  # Incorrectly identified

        tn = len(all_word_list) - (tp + fp + fn)
        metrics_mapping.append({"tp":tp, "fp":fp, "fn":fn, "tn":tn})
    return metrics_mapping


In [596]:
metrics_mapping = get_measures(true_sensitive_values, predicted_sensitive_values, all_words)

Missed entity (False Negative): Austin
Missed entity (False Negative): Randy
Missed entity (False Negative): Patti S
Missed entity (False Negative): Tuesday
Missed entity (False Negative): Philip
Missed entity (False Negative): Greg
Incorrectly identified (False Positive): Phillip
Missed entity (False Negative): Phillip Allen
Missed entity (False Negative): Mike Grigsby
Missed entity (False Negative): Keith Holst
Missed entity (False Negative): Monique Sanchez
Missed entity (False Negative): Frank Ermis
Missed entity (False Negative): John Lavorato
Incorrectly identified (False Positive): Monique
Incorrectly identified (False Positive): Sanchez
Incorrectly identified (False Positive): Frank
Incorrectly identified (False Positive): Ermis
Incorrectly identified (False Positive): John
Incorrectly identified (False Positive): Lavorato
Incorrectly identified (False Positive): Phillip
Incorrectly identified (False Positive): Allen
Missed entity (False Negative): 10
Missed entity (False Negat

In [597]:
def evaluate_model(metrics_mapping):
    # Initialize totals
    total_tp = 0
    total_fp = 0
    total_fn = 0
    total_tn = 0

    # Sum up the tp, fp, fn, tn for all sentences
    for metrics in metrics_mapping:
        total_tp += metrics["tp"]
        total_fp += metrics["fp"]
        total_fn += metrics["fn"]
        total_tn += metrics["tn"]

    # Compute the metrics
    accuracy = (total_tp + total_tn) / (total_tp + total_fp + total_fn + total_tn)
    precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
    recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    # Print the metrics
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1_score:.4f}")


In [598]:
evaluate_model(metrics_mapping)

Accuracy: 0.8937
Precision: 0.2363
Recall: 0.2384
F1 Score: 0.2374


In [101]:
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="average")
sentence = clean_text(df_filtered['source_text'][9])

results = ner_pipeline(sentence)

# Print results
for entity in results:
    print(f"Entity: {entity['word']}, Label: {entity['entity_group']}, Score: {entity['score']:.4f}")

Device set to use cpu


Entity: Apr29,1973, Label: DOB, Score: 0.9389
Entity: 29summikota, Label: DATE, Score: 0.4776
Entity: +51-063-367.7939, Label: PHONENUMBER, Score: 0.9291
Entity: May26,1993, Label: DOB, Score: 0.8120
Entity: npvhxlrgvjdhzjaf439498, Label: PASSWORD, Score: 0.8108
Entity: +20-914.834.1296, Label: PHONENUMBER, Score: 0.7936


In [102]:
true_sensitive_values[9]

['Apr29,1973',
 '29summikota',
 '+51-063-367.7939',
 'May26,1993',
 'npvhxlrgvjdhzjaf439498',
 '+20-914.834.1296']

### Dataset - email dataset labelled

Samples = 10

Accuracy: 0.9121
Precision: 0.4419
Recall: 0.3878
F1 Score: 0.4130


Samples = 100

Accuracy: 0.8937
Precision: 0.2363
Recall: 0.2384
F1 Score: 0.2374

### Dataset - pii-masking-300k

Samples = 10 

Accuracy: 0.9495, 
Precision: 0.7647,
Recall: 0.8125,
F1 Score: 0.7879


Samples = 100

Accuracy: 0.9329,
Precision: 0.7618,
Recall: 0.6232,
F1 Score: 0.6856


Samples = 1000

Accuracy: 0.9278,
Precision: 0.7608,
Recall: 0.5916,
F1 Score: 0.6656

Prediction using Ollama models 

In [342]:
import json

In [620]:
source_text_file_name = "source_texts.json"
predicted_labels_file_name = "outputs/predicted_labels_qwen_new_1000.json"
model_name = 'qwen2.5:3b'

In [616]:
#collect source texts - 

def collect_source_texts(data, file_name):
    source_texts = data['source_text'].tolist()
    with open(file_name, "w") as json_file:
        json.dump(source_texts, json_file)

In [617]:
collect_source_texts(df_filtered, source_text_file_name)

In [618]:
import json

def retrieve_source_texts(file_name):
    with open(file_name, "r") as json_file:
        source_texts = json.load(json_file)
    return source_texts

def count_source_texts(file_name):
    source_texts = retrieve_source_texts(file_name)
    return len(source_texts)

# Example usage
file_name = source_text_file_name
source_text_count = count_source_texts(file_name)
print(f"Number of source texts: {source_text_count}")


Number of source texts: 1000


In [619]:
!node evaluation.js {model_name} {source_text_file_name} {predicted_labels_file_name}

processSourceTexts() called
Prompt 0 processing time 107.4474136 seconds
Prompt 1 processing time 16.874359100000003 seconds
Prompt 2 processing time 18.807708299999998 seconds
Prompt 3 processing time 33.58980710000001 seconds
Prompt 4 processing time 20.908436400000006 seconds
Prompt 5 processing time 15.157294599999993 seconds
Prompt 6 processing time 23.343174300000012 seconds
Prompt 7 processing time 13.898001399999979 seconds
Prompt 8 processing time 25.619633299999986 seconds
Prompt 9 processing time 27.138930399999953 seconds
Prompt processing time for batch = 302.7847584999999
Batch 0 to 9 processed and saved successfully.
Prompt 0 processing time 24.997594100000047 seconds
Prompt 1 processing time 16.260090600000055 seconds
Prompt 2 processing time 17.5682083 seconds
Prompt 3 processing time 39.07036659999995 seconds
Prompt 4 processing time 41.77763349999999 seconds
Prompt 5 processing time 18.22193109999999 seconds
Prompt 6 processing time 39.5487942 seconds
Prompt 7 proces

In [621]:
def get_predicted_labels(file_name):
    with open(file_name, 'r', encoding='utf-8') as file:
        data = json.load(file)  # Load the entire JSON object

    # Extract results and average processing time
    raw_results = data.get("results", [])  # Default to empty list if missing
    total_time = float(data.get("processingTimeSeconds", 0))  # Convert string to float
    average_time_per_prompt = float(data.get("averageTimePerPromptSeconds", 0))

    # Convert JSON strings into dictionaries
    parsed_results = []
    for item in raw_results:
        if item is not None:  
            try:
                parsed_results.append(json.loads(item))
            except json.JSONDecodeError:
                print(f"Invalid JSON detected, storing as empty object: {item}")
                parsed_results.append({})
        else:
            print("Null item found, storing as empty object")
            parsed_results.append({})

    return parsed_results, total_time, average_time_per_prompt

In [622]:
predicted_labels, total_time, average_time_per_prompt = get_predicted_labels(predicted_labels_file_name)

Null item found, storing as empty object
Null item found, storing as empty object
Null item found, storing as empty object
Null item found, storing as empty object
Null item found, storing as empty object
Null item found, storing as empty object
Null item found, storing as empty object
Null item found, storing as empty object
Null item found, storing as empty object
Null item found, storing as empty object
Null item found, storing as empty object
Null item found, storing as empty object
Null item found, storing as empty object
Null item found, storing as empty object
Null item found, storing as empty object
Null item found, storing as empty object
Null item found, storing as empty object
Null item found, storing as empty object
Null item found, storing as empty object
Null item found, storing as empty object
Null item found, storing as empty object
Null item found, storing as empty object
Null item found, storing as empty object
Null item found, storing as empty object
Null item found,

In [623]:
len(predicted_labels)

1000

In [624]:
predicted_labels

[{'Subject': 'Group Messaging for Admissions Process',
  'username': 'pdmjrsyoz1460',
  'meeting_time': '9:47 PM',
  'firstname.lastname': 'luka.burg',
  'fullname': 'gholamhossein.ruschke'},
 {'name': 'N23',
  'Meeting at 11:29pm': 'Meeting at 11:29pm',
  'username': 'wennmann27',
  'Meeting at 4:45 PM': 'Meeting at 4:45 PM',
  'meetingTime': 'Meeting at 2:33 PM'},
 {'Subject': 'Admission Notification - Great Britain University',
  'Date of Notification': '5:24am on August 5th, 2057',
  'Name': 'Balloi Eckrich',
  'Email': 'bballoi@yahoo.com',
  'Social Number': '996 076 6460',
  'ID': 'ID'},
 {'Card': 'KB90324ER',
  'Country': 'United Kingdom',
  'Building': '646',
  'Street': 'School Lane',
  'City': 'Altrincham',
  'State': 'ENG',
  'Postcode': 'WA14 5R',
  'Password': 'q4R\\\\',
  'Applicant': 'Baasgaran Palmoso',
  'Email': 'blerenbaasgara@gmail.com',
  'Social Number': '107-393-9036',
  'ID Card': 'SC78428CU'},
 {'name': 'N', 'lastname': 'WA14 5RW', 'password': 'r]iD1#8'},
 {'pa

In [625]:
predicted_sensitive_values = []

In [626]:
predicted_sensitive_values = [list(d.values()) for d in predicted_labels]

In [627]:
predicted_sensitive_values

[['Group Messaging for Admissions Process',
  'pdmjrsyoz1460',
  '9:47 PM',
  'luka.burg',
  'gholamhossein.ruschke'],
 ['N23',
  'Meeting at 11:29pm',
  'wennmann27',
  'Meeting at 4:45 PM',
  'Meeting at 2:33 PM'],
 ['Admission Notification - Great Britain University',
  '5:24am on August 5th, 2057',
  'Balloi Eckrich',
  'bballoi@yahoo.com',
  '996 076 6460',
  'ID'],
 ['KB90324ER',
  'United Kingdom',
  '646',
  'School Lane',
  'Altrincham',
  'ENG',
  'WA14 5R',
  'q4R\\\\',
  'Baasgaran Palmoso',
  'blerenbaasgara@gmail.com',
  '107-393-9036',
  'SC78428CU'],
 ['N', 'WA14 5RW', 'r]iD1#8'],
 ['301025226', 'ROSALE 955306 9'],
 ['0610780437', 'xwjhgbgg009@outlook.com', '+543 51-082.8035'],
 [],
 ['Details',
  'Feb 8, 1986',
  '4929-667-4889',
  'Details',
  'abdi',
  '076 1352.8018'],
 ['Possesses extensive experience in the field of education, fostering a passion for nurturing young minds and shaping future generations through impactful teaching methodologies.',
  'May 26, 1993',


In [628]:
true_sensitive_values

[['wynqvrh053',
  '10:20am',
  'luka.burg',
  '21',
  'qahil.wittauer',
  'quarterpast13',
  'gholamhossein.ruschke',
  '9:47PM',
  'pdmjrsyoz1460'],
 ['2:33PM', 'N23', '11:29pm', 'wennmann27', '4:45PM'],
 ['5:24am',
  'August5th,2057',
  'Balloi',
  'Eckrich',
  'bballoi@yahoo.com',
  '9960766460'],
 ['KB90324ER',
  'GB',
  '163',
  'ConygreGrove',
  'Bristol',
  'ENG',
  'BS347HU,BS347HZ',
  'q4R\\\\',
  'Baasgaran',
  'Palmoso',
  'blerenbaasgara@gmail.com',
  '107-393-9036',
  'SC78428CU',
  'UnitedKingdom',
  '646',
  'SchoolLane',
  'Altrincham',
  'ENG'],
 ['r]iD1#8', 'August5th,2057'],
 ['301025226'],
 ['554.575.9355',
  'vtpkbqcutaxb799@yahoo.com',
  '+13223804181',
  '933932951',
  'LOUMA.657200.9.504',
  '0610780437',
  'xwjhgbgg009@outlook.com',
  '+54351-082.8035'],
 ['RuedesÉcoles', '13', 'September/54'],
 ['Feb8,1986',
  'iloweintögl',
  '4929-667-4889',
  '26/10/2004',
  'abdi',
  '0761352.8018'],
 ['Apr29,1973',
  '29summikota',
  '+51-063-367.7939',
  'May26,1993',
  

In [629]:
len(true_sensitive_values)

1000

In [630]:
metrics_mapping = get_measures(true_sensitive_values, predicted_sensitive_values, all_words)

Missed entity (False Negative): wynqvrh053
Missed entity (False Negative): 10:20am
Missed entity (False Negative): 21
Missed entity (False Negative): qahil.wittauer
Missed entity (False Negative): quarterpast13
Missed entity (False Negative): 9:47PM
Incorrectly identified (False Positive): Group Messaging for Admissions Process
Incorrectly identified (False Positive): 9:47 PM
Missed entity (False Negative): 2:33PM
Missed entity (False Negative): 4:45PM
Incorrectly identified (False Positive): Meeting at 4:45 PM
Incorrectly identified (False Positive): Meeting at 2:33 PM
Missed entity (False Negative): August5th,2057
Missed entity (False Negative): 9960766460
Incorrectly identified (False Positive): Admission Notification - Great Britain University
Incorrectly identified (False Positive): 996 076 6460
Incorrectly identified (False Positive): ID
Missed entity (False Negative): GB
Missed entity (False Negative): 163
Missed entity (False Negative): ConygreGrove
Missed entity (False Negativ

In [631]:
evaluate_model(metrics_mapping)

Accuracy: 0.8927
Precision: 0.5775
Recall: 0.4374
F1 Score: 0.4978


#### Llama3.2:3b 

Samples = 10

Accuracy: 0.8631,
Precision: 0.3235,
Recall: 0.1719,
F1 Score: 0.2245

Samples = 100

Accuracy: 0.8839,
Precision: 0.5095,
Recall: 0.3011,
F1 Score: 0.3785

Samples = 1000

Accuracy: 0.8812,
Precision: 0.5166,
Recall: 0.3490,
F1 Score: 0.4166

#### Llama3.2 - new

Samples = 10

Accuracy: 0.8937
Precision: 0.5556
Recall: 0.3906
F1 Score: 0.4587

Samples = 100

Accuracy: 0.8954
Precision: 0.5794
Recall: 0.3994
F1 Score: 0.4728


Samples = 1000

Accuracy: 0.8840
Precision: 0.5306
Recall: 0.3947
F1 Score: 0.4527


#### Qwen2.5:3b 

Samples = 10

Accuracy: 0.8937
Precision: 0.5455
Recall: 0.4688
F1 Score: 0.5042

Samples = 100

Accuracy: 0.9009
Precision: 0.6131
Recall: 0.4235
F1 Score: 0.5010

Samples = 1000

Accuracy: 0.8927
Precision: 0.5775
Recall: 0.4374
F1 Score: 0.4978